In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [7]:
# Setting up directories

WORKINGDIR = Path(os.getcwd())
PROJECTROOT = WORKINGDIR.parents[1]

POVERTY_ESTIMATES_DIR = Path(PROJECTROOT, "data", "gathered-datasets", "poverty-estimates.csv")
COMPLETE_DATASET = Path(PROJECTROOT, "data", "geolocation","ph_cities_coordinates", "cities_municipalities_with_coordinates.csv")

In [8]:
ref_df = pd.read_csv(COMPLETE_DATASET)
df = pd.read_csv(POVERTY_ESTIMATES_DIR)

In [10]:
ref_df.drop(columns=["Unnamed: 0"], inplace=True)

In [12]:
df.head()

,psgc,province,municipality-city,2009-poverty-inc,2012-poverty-inc,2015-poverty-inc,2009-cov,2012-cov,2015-cov,2009-conf-int-lowerlimit,2009-conf-int-upper-limit,2012-conf-int-lowerlimit,2012-conf-int-upper-limit,2015-conf-int-lowerlimit,2015-conf-int-upper-limit
0,133901,Metropolitan Manila,Tondo,"2,9","3,1","5,6","27,5","20,4","10,7","1,6","4,2","2,1","4,2","4,6","6,6"
1,133902,Metropolitan Manila,Binondo,"1,0","1,5","3,5","68,0","41,9","22,8","0,0","2,2","0,5","2,5","2,2","4,8"
2,133903,Metropolitan Manila,Quiapo,"2,1","2,1","7,1","42,4","28,1","17,9","0,6","3,5","1,1","3,1","5,0","9,2"
3,133904,Metropolitan Manila,San Nicolas,"2,4","3,0","6,7","57,0","53,6","27,2","0,1","4,7","0,4","5,5","3,7","9,7"
4,133905,Metropolitan Manila,Santa Cruz,"1,7","1,5","4,7","32,5","19,5","12,7","0,8","2,5","1,0","2,0","3,7","5,7"


In [13]:
ref_df.head()

,psgc,province,city_municipality,concat,corr_code,geog_level,old_names,city_cls,income_cls,urban_rural,2015_pop,2015_pop_tag,2020_pop,2020_pop_tag,status,search_term,longitude,latitude
0,102801000,Ilocos Norte,Adams,Ilocos Norte|Adams,12801000.0,Mun,NaN,NaN,5th,NaN,"1,792",NaN,"2,189",NaN,NaN,"Adams, Ilocos Norte",120.903571,18.461311
1,102802000,Ilocos Norte,Bacarra,Ilocos Norte|Bacarra,12802000.0,Mun,NaN,NaN,3rd,NaN,"32,215",NaN,"33,496",NaN,NaN,"Bacarra, Ilocos Norte",120.610659,18.251738
2,102803000,Ilocos Norte,Badoc,Ilocos Norte|Badoc,12803000.0,Mun,NaN,NaN,3rd,NaN,"31,616",NaN,"32,530",NaN,NaN,"Badoc, Ilocos Norte",120.474089,17.926701
3,102804000,Ilocos Norte,Bangui,Ilocos Norte|Bangui,12804000.0,Mun,NaN,NaN,4th,NaN,"14,672",NaN,"15,019",NaN,NaN,"Bangui, Ilocos Norte",120.765756,18.536739
4,102805000,Ilocos Norte,City of Batac,Ilocos Norte|City of Batac,12805000.0,City,NaN,CC,5th,NaN,"55,201",NaN,"55,484",NaN,NaN,"City of Batac, Ilocos Norte",120.581750,18.037300


In [22]:
ref_df.tail()

,psgc,province,city_municipality,concat,corr_code,geog_level,old_names,city_cls,income_cls,urban_rural,2015_pop,2015_pop_tag,2020_pop,2020_pop_tag,status,search_term,longitude,latitude
1629,1908820000,Maguindanao del Sur,Shariff Aguak,Maguindanao del Sur|Shariff Aguak,153808000.0,Mun,Maganoy,NaN,2nd,NaN,"31,692",NaN,"33,982",NaN,NaN,"Shariff Aguak, Maguindanao del Sur",124.444487,6.861017
1630,1908821000,Maguindanao del Sur,Shariff Saydona Mustapha,Maguindanao del Sur|Shariff Saydona Mustapha,153837000.0,Mun,NaN,NaN,-,NaN,"19,855",NaN,"25,080",NaN,NaN,"Shariff Saydona Mustapha, Maguindanao del Sur",124.484400,6.881631
1631,1908822000,Maguindanao del Sur,South Upi,Maguindanao del Sur|South Upi,153817000.0,Mun,NaN,NaN,4th*,NaN,"40,178",NaN,"43,197",NaN,NaN,"South Upi, Maguindanao del Sur",124.144000,6.854941
1632,1908823000,Maguindanao del Sur,Sultan Sa Barongis,Maguindanao del Sur|Sultan Sa Barongis,153813000.0,Mun,Lambayong,NaN,2nd,NaN,"22,425",NaN,"24,476",NaN,NaN,"Sultan Sa Barongis, Maguindanao del Sur",124.600205,6.882661
1633,1908824000,Maguindanao del Sur,Talayan,Maguindanao del Sur|Talayan,153816000.0,Mun,NaN,NaN,4th*,NaN,"30,032",NaN,"34,156",NaN,NaN,"Talayan, Maguindanao del Sur",124.356023,6.984152


In [21]:
ref_df["psgc"]

0        102801000
1        102802000
2        102803000
3        102804000
4        102805000
           ...    
1629    1908820000
1630    1908821000
1631    1908822000
1632    1908823000
1633    1908824000
Name: psgc, Length: 1634, dtype: int64